In [6]:
import pandas as pd

In [7]:
# Import the weather dataset
df = pd.read_csv("datasets/weather_data.csv")

In [9]:
df.head()
df.describe()

,year,month,day,temp_high,temp_avg,temp_low,humidity_high,humidity_avg,humidity_low
count,1091.000000,1091.000000,1091.000000,1091.000000,1091.000000,1091.000000,1091.000000,1091.000000,1091.000000
mean,2016.994500,6.538038,15.713107,17.537122,13.804766,9.813016,79.297892,62.695692,45.569203
std,0.816103,3.450192,8.820285,10.044545,9.571911,9.312212,16.005337,14.779262,16.129736
min,2016.000000,1.000000,1.000000,-11.000000,-14.000000,-18.000000,31.000000,22.000000,9.000000
25%,2016.000000,4.000000,8.000000,9.000000,6.000000,3.000000,66.500000,51.000000,34.000000
50%,2017.000000,7.000000,16.000000,18.000000,14.000000,9.000000,82.000000,63.000000,43.000000
75%,2018.000000,10.000000,23.000000,27.000000,23.000000,18.000000,93.000000,74.000000,55.000000
max,2018.000000,12.000000,31.000000,36.000000,32.000000,27.000000,100.000000,98.000000,96.000000
